<a href="https://colab.research.google.com/github/NguyenDacCuong2604/KLTN_2024/blob/main/Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/MyDrive/Thesis'

Mounted at /content/gdrive
/content/gdrive/MyDrive/Thesis


In [2]:
!pip install gensim

In [3]:
import pandas as pd

Dữ liệu vector hóa với vector_size là max_lenth

In [26]:
path_input = '/content/gdrive/MyDrive/Thesis/dataset/PreprocessedData/data_2_nam_keep_multi_label_and_not_remove_np.csv'
min_count = 3 #Loại bỏ các từ xuất hiện ít hơn một ngưỡng nhất định

In [27]:
df = pd.read_csv(path_input)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24618 entries, 0 to 24617
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Trích yếu       24618 non-null  object
 1   ID phòng xử lý  24618 non-null  int64 
 2   input           24618 non-null  object
dtypes: int64(1), object(2)
memory usage: 577.1+ KB


In [28]:
df['input'].fillna('', inplace=True)

In [69]:
word_counts= {}
for text in df['input']:
    words = text.split()
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

In [70]:
filtered_words = {word for word, count in word_counts.items() if count >= min_count}

In [71]:
df['input'] = df['input'].apply(lambda x: ' '.join([word for word in x.split() if word in filtered_words]))

In [72]:
vector_size = len(filtered_words)
print(vector_size)

2593


# TF-IDF

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [74]:
X = df['input']
X_transformed = tfidf.fit_transform(X)

In [75]:
df_tfidf = pd.DataFrame(X_transformed.toarray())

In [76]:
df_tfidf['label'] = df['ID phòng xử lý']

In [77]:
df_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24618 entries, 0 to 24617
Columns: 2594 entries, 0 to label
dtypes: float64(2593), int64(1)
memory usage: 487.2 MB


In [78]:
path_output = '/content/gdrive/MyDrive/Thesis/dataset/VectorHoa/TFIDF/Data2NamKeepMultiLabelAndNotRemoveNP'
name_file_output = 'data_vector_hoa.csv'

import os
if not os.path.exists(path_output):
    os.makedirs(path_output)

# Ghi DataFrame all_data vào file CSV
df_tfidf.to_csv(os.path.join(path_output, name_file_output), index=False)

In [79]:
#Lưu model vector hóa
import joblib
name_file_vector_hoa = 'tfidf_model.joblib'
joblib.dump(tfidf, os.path.join(path_output, name_file_vector_hoa))

['/content/gdrive/MyDrive/Thesis/dataset/VectorHoa/TFIDF/Data2NamKeepMultiLabelAndNotRemoveNP/tfidf_model.joblib']

# Doc2Vec

In [80]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import Counter
# Tạo danh sách các tài liệu được đánh nhãn
tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(df['input'])]

In [81]:
alpha = 0.025
min_alpha = 0.00025
epochs = 20

In [82]:
# Khởi tạo và huấn luyện mô hình PV-DBOW
model_doc2vec = Doc2Vec(vector_size=vector_size, alpha=alpha, min_alpha=min_alpha, min_count=min_count, dm=0)

In [83]:
# Xây dựng từ vựng từ dữ liệu được gắn thẻ
model_doc2vec.build_vocab(tagged_data)

In [84]:
# Huấn luyện mô hình
model_doc2vec.train(tagged_data, total_examples=model_doc2vec.corpus_count, epochs=epochs)

In [85]:
# Lấy vector cho mỗi tài liệu
doc_vectors = [model_doc2vec.dv[str(i)] for i in range(len(df))]
df_doc2vec = pd.DataFrame(doc_vectors)
df_doc2vec['label'] = df['ID phòng xử lý']

In [86]:
df_doc2vec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24618 entries, 0 to 24617
Columns: 2594 entries, 0 to label
dtypes: float32(2593), int64(1)
memory usage: 243.7 MB


In [87]:
path_output = '/content/gdrive/MyDrive/Thesis/dataset/VectorHoa/Doc2Vec/Data2NamKeepMultiLabelAndNotRemoveNP'
name_file_output = 'data_vector_hoa.csv'

import os
if not os.path.exists(path_output):
    os.makedirs(path_output)

# Ghi DataFrame all_data vào file CSV
df_doc2vec.to_csv(os.path.join(path_output, name_file_output), index=False)

In [88]:
#Lưu model vector hóa
model_doc2vec.save(os.path.join(path_output, 'model_doc2vec.model'))

# Word2Vec

In [89]:
sentences = df['input'].tolist()

In [91]:
from gensim.models.doc2vec import Word2Vec
#sử dụng skip-grams
model_word2vec = Word2Vec(sentences, vector_size=vector_size, window=5, sg=1)

In [99]:
import numpy as np
def vectorize_input(input_text, model):
    vectorized_input = []
    for sentence in input_text:
        # Lấy vector trung bình của các từ trong câu
        vectors = [model.wv[word] for word in sentence if word in model.wv]
        if vectors:
            vector = np.mean(vectors, axis=0)
        else:
            vector = np.zeros(model.vector_size)
        vectorized_input.append(vector)
    return vectorized_input

In [100]:
vectorized_input = vectorize_input(sentences, model_word2vec)

In [101]:
df_word2vec = pd.DataFrame(vectorized_input)

In [102]:
df_word2vec['label'] = df['ID phòng xử lý']
df_word2vec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24618 entries, 0 to 24617
Columns: 2594 entries, 0 to label
dtypes: float64(2593), int64(1)
memory usage: 487.2 MB


In [103]:
path_output = '/content/gdrive/MyDrive/Thesis/dataset/VectorHoa/Word2Vec/Data2NamKeepMultiLabelAndNotRemoveNP'
name_file_output = 'data_vector_hoa.csv'

import os
if not os.path.exists(path_output):
    os.makedirs(path_output)

# Ghi DataFrame all_data vào file CSV
df_word2vec.to_csv(os.path.join(path_output, name_file_output), index=False)

In [105]:
#Lưu model vector hóa
model_word2vec.save(os.path.join(path_output, 'model_word2vec.model'))